# Baseline performance of random/...

In [1]:
import numpy as np
import datasets
from tqdm.autonotebook import tqdm, trange
from matplotlib import pyplot as plt

In [18]:
pairs = datasets.load_dataset('0jl/SPair-71k', 'pairs', split='test', trust_remote_code=True).to_list()

In [ ]:
pairs[0]

In [ ]:
bbox_sizes = np.linspace(0, 0.15, 16)
correct_random = [[] for _ in range(len(bbox_sizes))]
correct_same_pos = [[] for _ in range(len(bbox_sizes))]
rng = np.random.default_rng(seed=42)
for pair in tqdm(pairs):
    trg_max_x, trg_max_y, _ = pair['trg_imsize']
    tbb_max = max(pair['trg_bndbox'][2] - pair['trg_bndbox'][0], pair['trg_bndbox'][3] - pair['trg_bndbox'][1])
    for (sx, sy), (tx, ty) in zip(pair['src_kps'], pair['trg_kps']):
        px, py = rng.random((2,1000)) * [[trg_max_x], [trg_max_y]]
        for i, bbox_size in enumerate(bbox_sizes):
            correct_random[i].append((np.sqrt((px - tx)**2 + (py - ty)**2) < tbb_max * bbox_size).mean())
            correct_same_pos[i].append(np.sqrt((sx - tx)**2 + (sy - ty)**2) < tbb_max * bbox_size)

print(f'BBox fraction: {bbox_sizes}')
print(f'Random: {np.mean(correct_random, axis=1)}')
print(f'Same position: {np.mean(correct_same_pos, axis=1)}')


In [ ]:
l1, = plt.plot(bbox_sizes, np.mean(correct_random, axis=1)*100)
l2, = plt.plot(bbox_sizes, np.mean(correct_same_pos, axis=1)*100)
plt.scatter(0.1, np.mean(correct_random[10])*100)
plt.scatter(0.1, np.mean(correct_same_pos[10])*100)
plt.text(0.105, np.mean(correct_random[10])*100, f'{np.mean(correct_random[10])*100:.2f}%', color=l1.get_color(), verticalalignment='top')
plt.text(0.105, np.mean(correct_same_pos[10])*100, f'{np.mean(correct_same_pos[10])*100:.2f}%', color=l2.get_color(), verticalalignment='top')

plt.xlim(0)
plt.xlabel('$\\alpha_{bbox}$')
plt.ylabel('PCK@$\\alpha_{bbox}$')
plt.legend([l2, l1], ['Same position', 'Random'])
plt.show()
